In [1]:
!pip install midiutil
!pip install pydub numpy
!apt-get install ffmpeg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for midiutil: filename=MIDIUtil-1.2.1-py3-none-any.whl size=54569 sha256=e013b5d4103a4b770ddc7cf90f8d3d26b3f60913c396fdad0627dc0ddf9ce571
  Stored in directory: /root/.cache/pip/wheels/6c/42/75/fce10c67f06fe627fad8acd1fd3a004a24e07b0f077761fbbd
Successfully built midiutil
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [2]:
from midiutil import MIDIFile
import os
import subprocess
import numpy as np
from scipy.io import wavfile
from pathlib import Path
from google.colab import drive
import time as time_module  # Renamed to avoid conflict

# Mount Google Drive
drive.mount('/content/drive')

# Define the paths
base_dir = '/content/drive/MyDrive/MusicGenerator'
midi_dir = os.path.join(base_dir, "midi")
wav_dir = os.path.join(base_dir, "wav")

# Create directories if they don't exist
chord_type = "major"
midi_chord_dir = os.path.join(midi_dir, "chords", chord_type)
wav_chord_dir = os.path.join(wav_dir, "chords", chord_type)

# Create a separate directory for the extreme tin noise files
extreme_tin_dir = os.path.join(wav_dir, "extreme_tin", chord_type)

# Create necessary directories
os.makedirs(midi_chord_dir, exist_ok=True)
os.makedirs(wav_chord_dir, exist_ok=True)
os.makedirs(extreme_tin_dir, exist_ok=True)

print(f"Created MIDI directory: {midi_chord_dir}")
print(f"Created WAV directory: {wav_chord_dir}")
print(f"Created EXTREME TIN directory: {extreme_tin_dir}")

# Install required packages if needed
try:
    import scipy
except ImportError:
    print("Installing scipy...")
    subprocess.run(['pip', 'install', 'scipy'], check=True)
    from scipy.io import wavfile

# Make sure midiutil is installed
try:
    from midiutil import MIDIFile
except ImportError:
    print("Installing midiutil...")
    subprocess.run(['pip', 'install', 'midiutil'], check=True)
    from midiutil import MIDIFile

noteArray = ["C", "C#", "D", "Eb", "E", "F", "F#", "G", "Ab", "A", "Bb", "B"]
octaveArray = [1, 2, 3, 4, 5, 6, 7, 8]

# Generate note names with octaves (C-1, C-2, etc.)
nameArray = []
for octave in octaveArray:
    for note in noteArray:
        nameArray.append(f"{note}-{octave}")

# Major triad parameters
offset1 = 4  # 4 semitones from root to third in a major triad
offset2 = 3  # 3 semitones from third to fifth in a major triad

# Function to generate EXTREME metallic tin sound
def generate_tin_sound(length, sample_rate=16000):
    """
    Generate an EXTREME metallic tin sound
    length: number of samples to generate
    sample_rate: sample rate of the audio (default 16kHz)
    Returns: array containing harsh tin sound
    """
    # Create time array
    t = np.linspace(0, length/sample_rate, length)
    tin_sound = np.zeros(length)

    # HARSH tin sound characteristics with extreme frequencies
    tin_frequencies = [
        1800, 2500, 3200,  # Lower metallic frequencies
        4300, 5100, 6200,  # Mid metallic frequencies
        7400, 8500, 9900, 11200  # High metallic frequencies for harsh effect
    ]

    # Add VERY HARSH overtones with minimal decay
    for freq in tin_frequencies:
        # Much slower decay for persistent tin sound
        decay_rate = 3 + (freq / 10000)  # MINIMAL decay
        envelope = np.exp(-decay_rate * t)

        # Add harsh metallic beating
        vibrato_rate = 30 + np.random.rand() * 20
        vibrato = 1.0 + 0.05 * np.sin(2 * np.pi * vibrato_rate * t)  # 10x stronger vibrato

        # Add strong distortion effect to the tin sound
        distortion_factor = 1.5  # Adds harmonic distortion
        clean_signal = np.sin(2 * np.pi * freq * t * vibrato)
        distorted_signal = np.tanh(distortion_factor * clean_signal)

        # Create component and add to overall sound with high amplitude
        component = envelope * distorted_signal
        tin_sound += component * 0.4  # Very high component scaling

    # Add random high-frequency noise for extra metallic character
    white_noise = np.random.normal(0, 0.3, length)
    # High-pass filter the noise (crude implementation)
    for i in range(3):  # Apply multiple times to strengthen high-pass effect
        white_noise[1:] = 0.8 * white_noise[1:] + 0.2 * white_noise[:-1]

    # Mix the tin components with the high-frequency noise
    tin_sound = tin_sound * 0.7 + white_noise * 0.3

    # Normalize but maintain high intensity
    tin_sound = tin_sound / (np.max(np.abs(tin_sound)) + 1e-10)

    return tin_sound

# Function for EXTREME tin sound overlay
def add_tin_noise(audio_data, noise_level):
    """
    Add EXTREME metallic tin background noise to audio
    noise_level: 0 to 5, where 0 is no noise and 5 is maximum noise
    Returns: audio with intense added tin noise
    """
    # DRASTICALLY increase amplitude based on noise level
    # Using exponential scaling for more dramatic effect at higher levels
    noise_amplitude = (noise_level / 5) ** 1.5 * 1.0  # Maximum 1.0 (same level as audio)

    # Convert to float64 for better precision during processing
    audio_float = audio_data.astype(np.float64)

    # Get the maximum value for the audio data type
    max_value = np.iinfo(audio_data.dtype).max
    audio_norm = audio_float / max_value  # Normalize to 0-1 range

    # Check if audio is stereo (2D array) or mono (1D array)
    is_stereo = len(audio_data.shape) > 1 and audio_data.shape[1] > 1

    # Generate tin sound noise - length matches audio
    if is_stereo:
        # Generate different tin sound for each channel for more immersive effect
        tin_sound_left = generate_tin_sound(audio_float.shape[0])
        tin_sound_right = generate_tin_sound(audio_float.shape[0])  # Different random seed
        # Create stereo tin noise
        tin_noise = np.column_stack([tin_sound_left, tin_sound_right]) * noise_amplitude
    else:
        # Generate mono tin sound
        tin_sound = generate_tin_sound(audio_float.shape[0])
        tin_noise = tin_sound * noise_amplitude

    # Mix signal and tin noise - using cross-fade method for stronger presence
    # At highest noise level, tin is 80% of the output, original sound is 50%
    # This creates overloud effect with clipping, intentionally for harsh sound
    mixed_audio = audio_norm * (1.0 - noise_amplitude * 0.5) + tin_noise * min(1.0, noise_amplitude * 1.6)

    # Convert back to original range and data type
    mixed_audio_scaled = mixed_audio * max_value
    mixed_audio_clipped = np.clip(mixed_audio_scaled, np.iinfo(audio_data.dtype).min, np.iinfo(audio_data.dtype).max)

    return mixed_audio_clipped.astype(audio_data.dtype)

def createTriad(inversion, base, offset1, offset2):
    if(inversion==0):
        note1 = base
        note2 = base+offset1
        note3 = note2+offset2

    elif(inversion==1):
        note1 = base+offset1
        note2 = note1+offset2
        note3 = base+12

    elif(inversion==2):
        note1 = base+offset1+offset2
        note2 = base+12
        note3 = note2+offset1

    return note1, note2, note3

print("Generating major triad chord MIDI files...")

# Counter for generated files
file_count = 0

# Limit to a subset of octaves to keep the number of generated files manageable
limited_octaves = [3, 4, 5]  # Using octaves 3-5

# Create a list to collect note information
note_list = []

for q, j in zip(nameArray, range(24,109)):
    # Get octave from the note name
    note_parts = q.split('-')
    if len(note_parts) == 2 and note_parts[1].isdigit():
        octave = int(note_parts[1])
        # Only process notes in our limited octave range
        if octave in limited_octaves:
            note_list.append((q, j))

# Process the collected notes
for noteName, base in note_list:
    for h in range(0, 3):  # Loop through all 3 inversions
        inversion = h

        appendInversion = "-"+str(inversion)

        note1, note2, note3 = createTriad(inversion, base, offset1, offset2)
        degrees = [note1, note2, note3]

        track    = 0
        channel  = 0
        time_pos = 0    # In beats (renamed from 'time' to avoid conflict)
        duration = 1    # In beats
        tempo    = 60   # In BPM
        volume   = 100  # 0-127, as per the MIDI standard

        MyMIDI = MIDIFile(1)  # One track
        MyMIDI.addTempo(track, time_pos, tempo)

        # Generate chord (all notes played simultaneously)
        for pitch in degrees:
            MyMIDI.addNote(track, channel, pitch, time_pos, duration, volume)

        midiName = noteName+'-'+chord_type+'-chord'+appendInversion+".mid"
        midi_filename = os.path.join(midi_chord_dir, midiName)

        with open(midi_filename, "wb") as output_file:
            MyMIDI.writeFile(output_file)
            file_count += 1

            # Print progress every 10 files
            if file_count % 10 == 0:
                print(f"Generated {file_count} MIDI files...")

print(f"Completed! Generated {file_count} major triad chord MIDI files.")

# Install timidity if not already installed
try:
    result = subprocess.run(['which', 'timidity'], capture_output=True, text=True)
    if result.returncode == 0:
        print("TiMidity++ is already installed.")
    else:
        raise subprocess.CalledProcessError(1, 'which timidity')
except (subprocess.CalledProcessError, FileNotFoundError):
    print("Installing TiMidity++...")
    subprocess.run(['apt-get', 'update'], check=True)
    subprocess.run(['apt-get', 'install', '-y', 'timidity'], check=True)
    print("TiMidity++ installed successfully.")

# Verify timidity is installed
try:
    subprocess.run(['timidity', '--version'], capture_output=True, check=True)
    print("TiMidity++ is working correctly.")
except (subprocess.CalledProcessError, FileNotFoundError):
    print("ERROR: TiMidity++ installation has issues. Please install manually.")
    raise RuntimeError("TiMidity++ not working properly")

# Convert MIDI files to WAV with different tin noise levels
print("\nConverting MIDI files to WAV format with varying tin noise levels (0-5)...")
wav_count = 0
extreme_wav_count = 0  # Counter for extreme tin files

# Get all MIDI files in the chord directory
midi_files = [f for f in os.listdir(midi_chord_dir) if f.endswith('.mid')]
total_midi_files = len(midi_files)
total_conversions = total_midi_files * 6  # 6 versions of each file (noise levels 0-5)

# Create temp directory if needed
temp_dir = '/tmp/midi_wav_temp'
os.makedirs(temp_dir, exist_ok=True)

for idx, midi_file in enumerate(midi_files):
    midi_path = os.path.join(midi_chord_dir, midi_file)
    base_name = os.path.splitext(midi_file)[0]

    print(f"Processing {idx+1}/{total_midi_files}: {midi_file}")

    # First, convert to WAV without noise (temp file)
    temp_wav_path = os.path.join(temp_dir, f"{base_name}_temp.wav")

    try:
        # Run timidity with a timeout to prevent hanging
        # Increase the volume of the MIDI files when converting to make them more prominent
        timidity_cmd = ['timidity', midi_path, '-Ow', '-s', '16000', '-A', '200', '-o', temp_wav_path]
        print(f"Running command: {' '.join(timidity_cmd)}")

        timidity_process = subprocess.run(
            timidity_cmd,
            stdout=subprocess.PIPE, stderr=subprocess.PIPE,
            timeout=30,  # Increased timeout for conversion
            text=True
        )

        print(f"TiMidity output: {timidity_process.stdout}")
        if timidity_process.stderr:
            print(f"TiMidity errors: {timidity_process.stderr}")

        # Check if the temp file was created
        if not os.path.exists(temp_wav_path):
            print(f"Warning: Failed to create WAV file for {midi_file}")
            print(f"Checking if temp directory exists: {os.path.exists(temp_dir)}")
            print(f"Directory contents: {os.listdir(temp_dir) if os.path.exists(temp_dir) else 'N/A'}")
            continue

        # Verify the file is properly created with non-zero size
        file_size = os.path.getsize(temp_wav_path)
        if file_size == 0:
            print(f"Warning: Zero-sized WAV file created for {midi_file}")
            continue
        else:
            print(f"Successfully created WAV file: {temp_wav_path} ({file_size} bytes)")

        # Read the WAV file
        try:
            sample_rate, audio_data = wavfile.read(temp_wav_path)
            print(f"Successfully read WAV file: Sample rate={sample_rate}, Shape={audio_data.shape}")
        except Exception as e:
            print(f"Error reading WAV file {temp_wav_path}: {str(e)}")
            continue

        # Create original versions in original directory (for reference)
        for noise_level in range(6):  # 0 to 5
            # Add appropriate level of tin noise - using original function if needed
            if noise_level == 0:
                # No noise for level 0
                noisy_audio = audio_data
            else:
                # Add noise using the original implementation
                # This is modified from the original code to use a much lower noise level
                # so there's a clear distinction between regular and extreme versions
                noise_amplitude = (noise_level / 5) * 0.01  # Original used 0.012

                # Convert to float64
                audio_float = audio_data.astype(np.float64)
                max_value = np.iinfo(audio_data.dtype).max

                # Generate basic tin sound (simplified version of original)
                t = np.linspace(0, len(audio_float)/sample_rate, len(audio_float))
                tin_component = np.sin(2 * np.pi * 4700 * t) * np.exp(-8 * t)

                # Mix at low level
                tin_noise = tin_component * max_value * noise_amplitude
                mixed_audio = audio_float + tin_noise

                # Clip and convert
                noisy_audio = np.clip(mixed_audio, np.iinfo(audio_data.dtype).min,
                                    np.iinfo(audio_data.dtype).max).astype(audio_data.dtype)

            # Save original version to regular directory for comparison
            wav_file = os.path.join(wav_chord_dir, f"{base_name}_tin_noise{noise_level}.wav")
            try:
                wavfile.write(wav_file, sample_rate, noisy_audio)
                print(f"Successfully wrote WAV file: {wav_file}")
                wav_count += 1
            except Exception as e:
                print(f"Error writing WAV file {wav_file}: {str(e)}")

        # Create extreme tin versions in the extreme tin directory
        for noise_level in range(6):  # 0 to 5
            # Use the extreme tin noise function
            if noise_level == 0:
                # No noise for level 0
                extreme_audio = audio_data
            else:
                # Add increasing levels of EXTREME tin noise
                extreme_audio = add_tin_noise(audio_data, noise_level)

                # Verify audio has meaningful content
                if np.max(np.abs(extreme_audio)) < 10:
                    print(f"Warning: Generated audio seems silent for {base_name} noise level {noise_level}")
                    # Force audible content in this case
                    tin_sound = generate_tin_sound(len(audio_data))
                    max_value = np.iinfo(audio_data.dtype).max
                    extreme_audio = (tin_sound * max_value * noise_level / 2).astype(audio_data.dtype)

            # Save to extreme tin directory with distinctive filename
            extreme_wav_file = os.path.join(extreme_tin_dir, f"{base_name}_EXTREME_tin{noise_level}.wav")
            try:
                wavfile.write(extreme_wav_file, sample_rate, extreme_audio)
                print(f"Successfully wrote extreme WAV file: {extreme_wav_file}")
                extreme_wav_count += 1
            except Exception as e:
                print(f"Error writing extreme WAV file {extreme_wav_file}: {str(e)}")

        # Success message for this file
        print(f"Created 6 regular and 6 EXTREME tin variations for {midi_file}")

    except subprocess.TimeoutExpired:
        print(f"Warning: Conversion timeout for {midi_file}")
    except Exception as e:
        print(f"Error processing {midi_file}: {str(e)}")
        import traceback
        traceback.print_exc()

    # Clean up temp file
    if os.path.exists(temp_wav_path):
        os.remove(temp_wav_path)

    # Add a small delay to prevent overwhelming the system
    time_module.sleep(0.2)  # Using the renamed time module

# Verify files were created
print("\nChecking for created files...")
try:
    midi_files_count = len([f for f in os.listdir(midi_chord_dir) if f.endswith('.mid')])
    wav_files_count = len([f for f in os.listdir(wav_chord_dir) if f.endswith('.wav')])
    extreme_files_count = len([f for f in os.listdir(extreme_tin_dir) if f.endswith('.wav')])

    print(f"MIDI files in {midi_chord_dir}: {midi_files_count}")
    print(f"WAV files in {wav_chord_dir}: {wav_files_count}")
    print(f"Extreme WAV files in {extreme_tin_dir}: {extreme_files_count}")
except Exception as e:
    print(f"Error checking directories: {str(e)}")

print(f"\nAll done! Generated {file_count} major triad chord MIDI files.")
print(f"{wav_count} regular WAV files with mild tin noise in: {wav_chord_dir}")
print(f"{extreme_wav_count} EXTREME tin noise WAV files in: {extreme_tin_dir}")

Mounted at /content/drive
Created MIDI directory: /content/drive/MyDrive/MusicGenerator/midi/chords/major
Created WAV directory: /content/drive/MyDrive/MusicGenerator/wav/chords/major
Created EXTREME TIN directory: /content/drive/MyDrive/MusicGenerator/wav/extreme_tin/major
Generating major triad chord MIDI files...
Generated 10 MIDI files...
Generated 20 MIDI files...
Generated 30 MIDI files...
Generated 40 MIDI files...
Generated 50 MIDI files...
Generated 60 MIDI files...
Generated 70 MIDI files...
Generated 80 MIDI files...
Generated 90 MIDI files...
Generated 100 MIDI files...
Completed! Generated 108 major triad chord MIDI files.
Installing TiMidity++...
TiMidity++ installed successfully.
TiMidity++ is working correctly.

Converting MIDI files to WAV format with varying tin noise levels (0-5)...
Processing 1/108: C-3-major-chord-0.mid
Running command: timidity /content/drive/MyDrive/MusicGenerator/midi/chords/major/C-3-major-chord-0.mid -Ow -s 16000 -A 200 -o /tmp/midi_wav_temp/C

Traceback (most recent call last):
  File "<ipython-input-2-067a161595b8>", line 341, in <cell line: 0>
    mixed_audio = audio_float + tin_noise
                  ~~~~~~~~~~~~^~~~~~~~~~~
ValueError: operands could not be broadcast together with shapes (48019,2) (48019,) 


Processing 2/108: C#-3-major-chord-1.mid
Running command: timidity /content/drive/MyDrive/MusicGenerator/midi/chords/major/C#-3-major-chord-1.mid -Ow -s 16000 -A 200 -o /tmp/midi_wav_temp/C#-3-major-chord-1_temp.wav
TiMidity output: Playing /content/drive/MyDrive/MusicGenerator/midi/chords/major/C#-3-major-chord-1.mid
MIDI file: /content/drive/MyDrive/MusicGenerator/midi/chords/major/C#-3-major-chord-1.mid
Format: 1  Tracks: 2  Divisions: 960
Playing time: ~5 seconds
Notes cut: 0
Notes lost totally: 0

Successfully created WAV file: /tmp/midi_wav_temp/C#-3-major-chord-1_temp.wav (192120 bytes)
Successfully read WAV file: Sample rate=16000, Shape=(48019, 2)
Successfully wrote WAV file: /content/drive/MyDrive/MusicGenerator/wav/chords/major/C#-3-major-chord-1_tin_noise0.wav
Error processing C#-3-major-chord-1.mid: operands could not be broadcast together with shapes (48019,2) (48019,) 


Traceback (most recent call last):
  File "<ipython-input-2-067a161595b8>", line 341, in <cell line: 0>
    mixed_audio = audio_float + tin_noise
                  ~~~~~~~~~~~~^~~~~~~~~~~
ValueError: operands could not be broadcast together with shapes (48019,2) (48019,) 


Processing 3/108: Eb-3-major-chord-0.mid
Running command: timidity /content/drive/MyDrive/MusicGenerator/midi/chords/major/Eb-3-major-chord-0.mid -Ow -s 16000 -A 200 -o /tmp/midi_wav_temp/Eb-3-major-chord-0_temp.wav
TiMidity output: Playing /content/drive/MyDrive/MusicGenerator/midi/chords/major/Eb-3-major-chord-0.mid
MIDI file: /content/drive/MyDrive/MusicGenerator/midi/chords/major/Eb-3-major-chord-0.mid
Format: 1  Tracks: 2  Divisions: 960
Playing time: ~5 seconds
Notes cut: 0
Notes lost totally: 0

Successfully created WAV file: /tmp/midi_wav_temp/Eb-3-major-chord-0_temp.wav (192120 bytes)
Successfully read WAV file: Sample rate=16000, Shape=(48019, 2)
Successfully wrote WAV file: /content/drive/MyDrive/MusicGenerator/wav/chords/major/Eb-3-major-chord-0_tin_noise0.wav
Error processing Eb-3-major-chord-0.mid: operands could not be broadcast together with shapes (48019,2) (48019,) 


...

Processing 107/108: B-5-major-chord-1.mid
Running command: timidity /content/drive/MyDrive/MusicGenerator/midi/chords/major/B-5-major-chord-1.mid -Ow -s 16000 -A 200 -o /tmp/midi_wav_temp/B-5-major-chord-1_temp.wav
TiMidity output: Playing /content/drive/MyDrive/MusicGenerator/midi/chords/major/B-5-major-chord-1.mid
MIDI file: /content/drive/MyDrive/MusicGenerator/midi/chords/major/B-5-major-chord-1.mid
Format: 1  Tracks: 2  Divisions: 960
Playing time: ~5 seconds
Notes cut: 0
Notes lost totally: 0

Successfully created WAV file: /tmp/midi_wav_temp/B-5-major-chord-1_temp.wav (192120 bytes)
Successfully read WAV file: Sample rate=16000, Shape=(48019, 2)
Successfully wrote WAV file: /content/drive/MyDrive/MusicGenerator/wav/chords/major/B-5-major-chord-1_tin_noise0.wav
Error processing B-5-major-chord-1.mid: operands could not be broadcast together with shapes (48019,2) (48019,) 


Traceback (most recent call last):
  File "<ipython-input-2-067a161595b8>", line 341, in <cell line: 0>
    mixed_audio = audio_float + tin_noise
                  ~~~~~~~~~~~~^~~~~~~~~~~
ValueError: operands could not be broadcast together with shapes (48019,2) (48019,) 


Processing 108/108: B-5-major-chord-2.mid
Running command: timidity /content/drive/MyDrive/MusicGenerator/midi/chords/major/B-5-major-chord-2.mid -Ow -s 16000 -A 200 -o /tmp/midi_wav_temp/B-5-major-chord-2_temp.wav
TiMidity output: Playing /content/drive/MyDrive/MusicGenerator/midi/chords/major/B-5-major-chord-2.mid
MIDI file: /content/drive/MyDrive/MusicGenerator/midi/chords/major/B-5-major-chord-2.mid
Format: 1  Tracks: 2  Divisions: 960
Playing time: ~5 seconds
Notes cut: 0
Notes lost totally: 0

Successfully created WAV file: /tmp/midi_wav_temp/B-5-major-chord-2_temp.wav (192120 bytes)
Successfully read WAV file: Sample rate=16000, Shape=(48019, 2)
Successfully wrote WAV file: /content/drive/MyDrive/MusicGenerator/wav/chords/major/B-5-major-chord-2_tin_noise0.wav
Error processing B-5-major-chord-2.mid: operands could not be broadcast together with shapes (48019,2) (48019,) 


Traceback (most recent call last):
  File "<ipython-input-2-067a161595b8>", line 341, in <cell line: 0>
    mixed_audio = audio_float + tin_noise
                  ~~~~~~~~~~~~^~~~~~~~~~~
ValueError: operands could not be broadcast together with shapes (48019,2) (48019,) 



Checking for created files...
MIDI files in /content/drive/MyDrive/MusicGenerator/midi/chords/major: 108
WAV files in /content/drive/MyDrive/MusicGenerator/wav/chords/major: 1296
Extreme WAV files in /content/drive/MyDrive/MusicGenerator/wav/extreme_tin/major: 0

All done! Generated 108 major triad chord MIDI files.
108 regular WAV files with mild tin noise in: /content/drive/MyDrive/MusicGenerator/wav/chords/major
0 EXTREME tin noise WAV files in: /content/drive/MyDrive/MusicGenerator/wav/extreme_tin/major


In [ ]:
#minor

from midiutil import MIDIFile
import os
import subprocess
import numpy as np
from scipy.io import wavfile
from pathlib import Path
from google.colab import drive
import time as time_module  # Renamed to avoid conflict

# Mount Google Drive
drive.mount('/content/drive')

# Define the paths
base_dir = '/content/drive/MyDrive/MusicGenerator'
midi_dir = os.path.join(base_dir, "midi")
wav_dir = os.path.join(base_dir, "wav")

# Create directories if they don't exist
chord_type = "minor"  # Changed from "major" to "minor"
midi_chord_dir = os.path.join(midi_dir, "chords", chord_type)
wav_chord_dir = os.path.join(wav_dir, "chords", chord_type)

# Create necessary directories
Path(midi_chord_dir).mkdir(parents=True, exist_ok=True)
Path(wav_chord_dir).mkdir(parents=True, exist_ok=True)

print(f"Created MIDI directory: {midi_chord_dir}")
print(f"Created WAV directory: {wav_chord_dir}")

# Install required packages if needed
try:
    import scipy
except ImportError:
    print("Installing scipy...")
    subprocess.run(['pip', 'install', 'scipy'], check=True)
    from scipy.io import wavfile

noteArray = ["C", "C#", "D", "Eb", "E", "F", "F#", "G", "Ab", "A", "Bb", "B"]
octaveArray = [1, 2, 3, 4, 5, 6, 7, 8]  # Using the full octave range 1-8

# Generate note names with octaves (C-1, C-2, etc.)
nameArray = []
for octave in octaveArray:
    for note in noteArray:
        nameArray.append(f"{note}-{octave}")

# Minor triad parameters
offset1 = 3  # 3 semitones from root to flat third in a minor triad (changed from 4)
offset2 = 4  # 4 semitones from flat third to fifth in a minor triad (changed from 3)

# Function to add background noise to audio while preserving original volume
def add_noise(audio_data, noise_level):
    """
    Add background noise to audio at specified level without reducing original volume
    noise_level: 0 to 5, where 0 is no noise and 5 is maximum noise
    Returns: audio with added noise, maintaining original signal strength
    """
    # Scale noise level (0-5) to amplitude
    # Values higher than 0.5 will create very noticeable noise
    noise_amplitude = (noise_level / 5) * 0.008

    # Convert to float64 for better precision during processing
    audio_float = audio_data.astype(np.float64)

    # Get the maximum value for the audio data type
    max_value = np.iinfo(audio_data.dtype).max

    # Check if audio is stereo (2D array) or mono (1D array)
    is_stereo = len(audio_data.shape) > 1 and audio_data.shape[1] > 1

    # Generate appropriate noise
    if is_stereo:
        # Handle stereo audio (separate noise for each channel)
        noise = np.random.normal(0, max_value * noise_amplitude, audio_data.shape).astype(np.float64)
    else:
        # Handle mono audio
        noise = np.random.normal(0, max_value * noise_amplitude, audio_data.shape[0]).astype(np.float64)

    # Calculate mixing ratio based on noise level
    # This ensures the original signal maintains its volume
    signal_weight = 1.0  # Keep original signal at 100%

    # Mix signal and noise with proper weighting
    mixed_audio = (signal_weight * audio_float) + noise

    # Clip to valid range for the data type
    mixed_audio_clipped = np.clip(mixed_audio, np.iinfo(audio_data.dtype).min, np.iinfo(audio_data.dtype).max)

    # Convert back to original data type
    return mixed_audio_clipped.astype(audio_data.dtype)

def createTriad(inversion, base, offset1, offset2):
    if(inversion==0):
        note1 = base
        note2 = base+offset1
        note3 = note2+offset2

    elif(inversion==1):
        note1 = base+offset1
        note2 = note1+offset2
        note3 = base+12

    elif(inversion==2):
        note1 = base+offset1+offset2
        note2 = base+12
        note3 = note2+offset1

    return note1, note2, note3

print("Generating minor triad chord MIDI files...")  # Changed from major to minor

# Counter for generated files
file_count = 0

# Process all notes in all octaves without limiting
for q, j in zip(nameArray, range(24,109)):
    noteName = q
    base = j

    for h in range(0, 3):  # Loop through all 3 inversions
        inversion = h
        appendInversion = "-"+str(inversion)

        note1, note2, note3 = createTriad(inversion, base, offset1, offset2)
        degrees = [note1, note2, note3]

        track    = 0
        channel  = 0
        time_pos = 0    # In beats (renamed from 'time' to avoid conflict)
        duration = 1    # In beats
        tempo    = 60   # In BPM
        volume   = 100  # 0-127, as per the MIDI standard

        MyMIDI = MIDIFile(1)  # One track
        MyMIDI.addTempo(track, time_pos, tempo)

        # Generate chord (all notes played simultaneously)
        for pitch in degrees:
            MyMIDI.addNote(track, channel, pitch, time_pos, duration, volume)

        midiName = noteName+'-'+chord_type+'-chord'+appendInversion+".mid"
        midi_filename = os.path.join(midi_chord_dir, midiName)

        with open(midi_filename, "wb") as output_file:
            MyMIDI.writeFile(output_file)
            file_count += 1

            # Print progress every 10 files
            if file_count % 10 == 0:
                print(f"Generated {file_count} MIDI files...")

print(f"Completed! Generated {file_count} minor triad chord MIDI files.")  # Changed from major to minor

# Install timidity if not already installed
try:
    subprocess.run(['which', 'timidity'], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print("TiMidity++ is already installed.")
except subprocess.CalledProcessError:
    print("Installing TiMidity++...")
    subprocess.run(['apt-get', 'update'], check=True)
    subprocess.run(['apt-get', 'install', '-y', 'timidity'], check=True)
    print("TiMidity++ installed successfully.")

# Convert MIDI files to WAV with different noise levels
print("\nConverting MIDI files to WAV format with varying noise levels (0-5)...")
wav_count = 0

# Get all MIDI files in the chord directory
midi_files = [f for f in os.listdir(midi_chord_dir) if f.endswith('.mid')]
total_midi_files = len(midi_files)
total_conversions = total_midi_files * 6  # 6 versions of each file (noise levels 0-5)

for idx, midi_file in enumerate(midi_files):
    midi_path = os.path.join(midi_chord_dir, midi_file)
    base_name = Path(midi_file).stem

    print(f"Processing {idx+1}/{total_midi_files}: {midi_file}")

    # First, convert to WAV without noise (temp file)
    temp_wav_path = os.path.join('/tmp', f"{base_name}_temp.wav")

    try:
        # Run timidity with a timeout to prevent hanging
        # Increase the volume of the MIDI files when converting to make them more prominent
        timidity_process = subprocess.run(
            ['timidity', midi_path, '-Ow1', '-s', '16kHz', '-A', '200', '-o', temp_wav_path],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE,
            timeout=10  # Set a 10-second timeout for conversion
        )

        # Check if the temp file was created
        if not os.path.exists(temp_wav_path):
            print(f"Warning: Failed to create WAV file for {midi_file}")
            continue

        # Read the WAV file
        sample_rate, audio_data = wavfile.read(temp_wav_path)

        # Create versions with different noise levels
        for noise_level in range(6):  # 0 to 5
            # Add appropriate level of noise
            if noise_level == 0:
                # No noise for level 0
                noisy_audio = audio_data
            else:
                # Add increasing levels of noise
                noisy_audio = add_noise(audio_data, noise_level)

            # Save to final WAV file with noise level in filename
            wav_file = os.path.join(wav_chord_dir, f"{base_name}_noise{noise_level}.wav")
            wavfile.write(wav_file, sample_rate, noisy_audio)
            wav_count += 1

        # Success message for this file
        print(f"Created 6 variations with different noise levels for {midi_file}")

    except subprocess.TimeoutExpired:
        print(f"Warning: Conversion timeout for {midi_file}")
    except Exception as e:
        print(f"Error processing {midi_file}: {str(e)}")

    # Clean up temp file
    if os.path.exists(temp_wav_path):
        os.remove(temp_wav_path)

    # Add a small delay to prevent overwhelming the system
    time_module.sleep(0.1)  # Using the renamed time module

print(f"\nAll done! Generated {file_count} minor triad chord MIDI files and {wav_count} WAV files with varying noise levels.")  # Changed from major to minor
print(f"MIDI files are in: {midi_chord_dir}")
print(f"WAV files are in: {wav_chord_dir}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Created MIDI directory: /content/drive/MyDrive/MusicGenerator/midi/chords/minor
Created WAV directory: /content/drive/MyDrive/MusicGenerator/wav/chords/minor
Generating minor triad chord MIDI files...
Generated 10 MIDI files...
Generated 20 MIDI files...
Generated 30 MIDI files...
Generated 40 MIDI files...
Generated 50 MIDI files...
Generated 60 MIDI files...
Generated 70 MIDI files...
Generated 80 MIDI files...
Generated 90 MIDI files...
Generated 100 MIDI files...
Generated 110 MIDI files...
Generated 120 MIDI files...
Generated 130 MIDI files...
Generated 140 MIDI files...
Generated 150 MIDI files...
Generated 160 MIDI files...
Generated 170 MIDI files...
Generated 180 MIDI files...
Generated 190 MIDI files...
Generated 200 MIDI files...
Generated 210 MIDI files...
Generated 220 MIDI files...
Generated 230 MIDI files...
Generated 240 MIDI files...
Generat

In [ ]:
#diminished

from midiutil import MIDIFile
import os
import subprocess
import numpy as np
from scipy.io import wavfile
from pathlib import Path
from google.colab import drive
import time as time_module  # Renamed to avoid conflict

# Mount Google Drive
drive.mount('/content/drive')

# Define the paths
base_dir = '/content/drive/MyDrive/MusicGenerator'
midi_dir = os.path.join(base_dir, "midi")
wav_dir = os.path.join(base_dir, "wav")

# Create directories if they don't exist
chord_type = "diminished"  # Changed to "diminished"
midi_chord_dir = os.path.join(midi_dir, "chords", chord_type)
wav_chord_dir = os.path.join(wav_dir, "chords", chord_type)

# Create necessary directories
Path(midi_chord_dir).mkdir(parents=True, exist_ok=True)
Path(wav_chord_dir).mkdir(parents=True, exist_ok=True)

print(f"Created MIDI directory: {midi_chord_dir}")
print(f"Created WAV directory: {wav_chord_dir}")

# Install required packages if needed
try:
    import scipy
except ImportError:
    print("Installing scipy...")
    subprocess.run(['pip', 'install', 'scipy'], check=True)
    from scipy.io import wavfile

noteArray = ["C", "C#", "D", "Eb", "E", "F", "F#", "G", "Ab", "A", "Bb", "B"]
octaveArray = [1, 2, 3, 4, 5, 6, 7, 8]  # Using the full octave range 1-8

# Generate note names with octaves (C-1, C-2, etc.)
nameArray = []
for octave in octaveArray:
    for note in noteArray:
        nameArray.append(f"{note}-{octave}")

# Diminished triad parameters
offset1 = 3  # 3 semitones from root to minor third
offset2 = 3  # 3 semitones from minor third to diminished fifth (changed from 4)

# Function to add background noise to audio while preserving original volume
def add_noise(audio_data, noise_level):
    """
    Add background noise to audio at specified level without reducing original volume
    noise_level: 0 to 5, where 0 is no noise and 5 is maximum noise
    Returns: audio with added noise, maintaining original signal strength
    """
    # Scale noise level (0-5) to amplitude
    # Values higher than 0.5 will create very noticeable noise
    noise_amplitude = (noise_level / 5) * 0.008

    # Convert to float64 for better precision during processing
    audio_float = audio_data.astype(np.float64)

    # Get the maximum value for the audio data type
    max_value = np.iinfo(audio_data.dtype).max

    # Check if audio is stereo (2D array) or mono (1D array)
    is_stereo = len(audio_data.shape) > 1 and audio_data.shape[1] > 1

    # Generate appropriate noise
    # Generate white noise instead of Gaussian noise
    if is_stereo:
        # White noise for stereo (same distribution for both channels)
        noise = np.random.uniform(-max_value * noise_amplitude, max_value * noise_amplitude, audio_data.shape).astype(np.float64)
    else:
        # White noise for mono
        noise = np.random.uniform(-max_value * noise_amplitude, max_value * noise_amplitude, audio_data.shape[0]).astype(np.float64)


    # Calculate mixing ratio based on noise level
    # This ensures the original signal maintains its volume
    signal_weight = 1.0  # Keep original signal at 100%

    # Mix signal and noise with proper weighting
    mixed_audio = (signal_weight * audio_float) + noise

    # Clip to valid range for the data type
    mixed_audio_clipped = np.clip(mixed_audio, np.iinfo(audio_data.dtype).min, np.iinfo(audio_data.dtype).max)

    # Convert back to original data type
    return mixed_audio_clipped.astype(audio_data.dtype)

def createTriad(inversion, base, offset1, offset2):
    if(inversion==0):
        note1 = base
        note2 = base+offset1
        note3 = note2+offset2

    elif(inversion==1):
        note1 = base+offset1
        note2 = note1+offset2
        note3 = base+12

    elif(inversion==2):
        note1 = base+offset1+offset2
        note2 = base+12
        note3 = note2+offset1

    return note1, note2, note3

print("Generating diminished triad chord MIDI files...")  # Changed to diminished

# Counter for generated files
file_count = 0

# Process all notes in all octaves without limiting
for q, j in zip(nameArray, range(24,109)):
    noteName = q
    base = j

    for h in range(0, 3):  # Loop through all 3 inversions
        inversion = h
        appendInversion = "-"+str(inversion)

        note1, note2, note3 = createTriad(inversion, base, offset1, offset2)
        degrees = [note1, note2, note3]

        track    = 0
        channel  = 0
        time_pos = 0    # In beats (renamed from 'time' to avoid conflict)
        duration = 1    # In beats
        tempo    = 60   # In BPM
        volume   = 100  # 0-127, as per the MIDI standard

        MyMIDI = MIDIFile(1)  # One track
        MyMIDI.addTempo(track, time_pos, tempo)

        # Generate chord (all notes played simultaneously)
        for pitch in degrees:
            MyMIDI.addNote(track, channel, pitch, time_pos, duration, volume)

        midiName = noteName+'-'+chord_type+'-chord'+appendInversion+".mid"
        midi_filename = os.path.join(midi_chord_dir, midiName)

        with open(midi_filename, "wb") as output_file:
            MyMIDI.writeFile(output_file)
            file_count += 1

            # Print progress every 10 files
            if file_count % 10 == 0:
                print(f"Generated {file_count} MIDI files...")

print(f"Completed! Generated {file_count} diminished triad chord MIDI files.")  # Changed to diminished

# Install timidity if not already installed
try:
    subprocess.run(['which', 'timidity'], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print("TiMidity++ is already installed.")
except subprocess.CalledProcessError:
    print("Installing TiMidity++...")
    subprocess.run(['apt-get', 'update'], check=True)
    subprocess.run(['apt-get', 'install', '-y', 'timidity'], check=True)
    print("TiMidity++ installed successfully.")

# Convert MIDI files to WAV with different noise levels
print("\nConverting MIDI files to WAV format with varying noise levels (0-5)...")
wav_count = 0

# Get all MIDI files in the chord directory
midi_files = [f for f in os.listdir(midi_chord_dir) if f.endswith('.mid')]
total_midi_files = len(midi_files)
total_conversions = total_midi_files * 6  # 6 versions of each file (noise levels 0-5)

for idx, midi_file in enumerate(midi_files):
    midi_path = os.path.join(midi_chord_dir, midi_file)
    base_name = Path(midi_file).stem

    print(f"Processing {idx+1}/{total_midi_files}: {midi_file}")

    # First, convert to WAV without noise (temp file)
    temp_wav_path = os.path.join('/tmp', f"{base_name}_temp.wav")

    try:
        # Run timidity with a timeout to prevent hanging
        # Increase the volume of the MIDI files when converting to make them more prominent
        timidity_process = subprocess.run(
            ['timidity', midi_path, '-Ow1', '-s', '16kHz', '-A', '200', '-o', temp_wav_path],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE,
            timeout=10  # Set a 10-second timeout for conversion
        )

        # Check if the temp file was created
        if not os.path.exists(temp_wav_path):
            print(f"Warning: Failed to create WAV file for {midi_file}")
            continue

        # Read the WAV file
        sample_rate, audio_data = wavfile.read(temp_wav_path)

        # Create versions with different noise levels
        for noise_level in range(6):  # 0 to 5
            # Add appropriate level of noise
            if noise_level == 0:
                # No noise for level 0
                noisy_audio = audio_data
            else:
                # Add increasing levels of noise
                noisy_audio = add_noise(audio_data, noise_level)

            # Save to final WAV file with noise level in filename
            wav_file = os.path.join(wav_chord_dir, f"{base_name}_noise{noise_level}.wav")
            wavfile.write(wav_file, sample_rate, noisy_audio)
            wav_count += 1

        # Success message for this file
        print(f"Created 6 variations with different noise levels for {midi_file}")

    except subprocess.TimeoutExpired:
        print(f"Warning: Conversion timeout for {midi_file}")
    except Exception as e:
        print(f"Error processing {midi_file}: {str(e)}")

    # Clean up temp file
    if os.path.exists(temp_wav_path):
        os.remove(temp_wav_path)

    # Add a small delay to prevent overwhelming the system
    time_module.sleep(0.1)  # Using the renamed time module

print(f"\nAll done! Generated {file_count} diminished triad chord MIDI files and {wav_count} WAV files with varying noise levels.")  # Changed to diminished
print(f"MIDI files are in: {midi_chord_dir}")
print(f"WAV files are in: {wav_chord_dir}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Created MIDI directory: /content/drive/MyDrive/MusicGenerator/midi/chords/diminished
Created WAV directory: /content/drive/MyDrive/MusicGenerator/wav/chords/diminished
Generating diminished triad chord MIDI files...
Generated 10 MIDI files...
Generated 20 MIDI files...
Generated 30 MIDI files...
Generated 40 MIDI files...
Generated 50 MIDI files...
Generated 60 MIDI files...
Generated 70 MIDI files...
Generated 80 MIDI files...
Generated 90 MIDI files...
Generated 100 MIDI files...
Generated 110 MIDI files...
Generated 120 MIDI files...
Generated 130 MIDI files...
Generated 140 MIDI files...
Generated 150 MIDI files...
Generated 160 MIDI files...
Generated 170 MIDI files...
Generated 180 MIDI files...
Generated 190 MIDI files...
Generated 200 MIDI files...
Generated 210 MIDI files...
Generated 220 MIDI files...
Generated 230 MIDI files...
Generated 240 MIDI f

KeyboardInterrupt: 

In [ ]:
#augmented

from midiutil import MIDIFile
import os
import subprocess
import numpy as np
from scipy.io import wavfile
from pathlib import Path
from google.colab import drive
import time as time_module  # Renamed to avoid conflict

# Mount Google Drive
drive.mount('/content/drive')

# Define the paths
base_dir = '/content/drive/MyDrive/MusicGenerator'
midi_dir = os.path.join(base_dir, "midi")
wav_dir = os.path.join(base_dir, "wav")

# Create directories if they don't exist
chord_type = "augmented"  # Changed to "augmented"
midi_chord_dir = os.path.join(midi_dir, "chords", chord_type)
wav_chord_dir = os.path.join(wav_dir, "chords", chord_type)

# Create necessary directories
Path(midi_chord_dir).mkdir(parents=True, exist_ok=True)
Path(wav_chord_dir).mkdir(parents=True, exist_ok=True)

print(f"Created MIDI directory: {midi_chord_dir}")
print(f"Created WAV directory: {wav_chord_dir}")

# Install required packages if needed
try:
    import scipy
except ImportError:
    print("Installing scipy...")
    subprocess.run(['pip', 'install', 'scipy'], check=True)
    from scipy.io import wavfile

noteArray = ["C", "C#", "D", "Eb", "E", "F", "F#", "G", "Ab", "A", "Bb", "B"]
octaveArray = [1, 2, 3, 4, 5, 6, 7, 8]  # Using the full octave range 1-8

# Generate note names with octaves (C-1, C-2, etc.)
nameArray = []
for octave in octaveArray:
    for note in noteArray:
        nameArray.append(f"{note}-{octave}")

# Augmented triad parameters
offset1 = 4  # 4 semitones from root to major third
offset2 = 4  # 4 semitones from major third to augmented fifth (changed from 3)

# Function to add background noise to audio while preserving original volume
def add_noise(audio_data, noise_level):
    """
    Add background noise to audio at specified level without reducing original volume
    noise_level: 0 to 5, where 0 is no noise and 5 is maximum noise
    Returns: audio with added noise, maintaining original signal strength
    """
    # Scale noise level (0-5) to amplitude
    # Values higher than 0.5 will create very noticeable noise
    noise_amplitude = (noise_level / 5) * 0.008

    # Convert to float64 for better precision during processing
    audio_float = audio_data.astype(np.float64)

    # Get the maximum value for the audio data type
    max_value = np.iinfo(audio_data.dtype).max

    # Check if audio is stereo (2D array) or mono (1D array)
    is_stereo = len(audio_data.shape) > 1 and audio_data.shape[1] > 1

    # Generate appropriate noise
    # Generate white noise instead of Gaussian noise
    if is_stereo:
        # White noise for stereo (same distribution for both channels)
        noise = np.random.uniform(-max_value * noise_amplitude, max_value * noise_amplitude, audio_data.shape).astype(np.float64)
    else:
        # White noise for mono
        noise = np.random.uniform(-max_value * noise_amplitude, max_value * noise_amplitude, audio_data.shape[0]).astype(np.float64)


    # Calculate mixing ratio based on noise level
    # This ensures the original signal maintains its volume
    signal_weight = 1.0  # Keep original signal at 100%

    # Mix signal and noise with proper weighting
    mixed_audio = (signal_weight * audio_float) + noise

    # Clip to valid range for the data type
    mixed_audio_clipped = np.clip(mixed_audio, np.iinfo(audio_data.dtype).min, np.iinfo(audio_data.dtype).max)

    # Convert back to original data type
    return mixed_audio_clipped.astype(audio_data.dtype)

def createTriad(inversion, base, offset1, offset2):
    if(inversion==0):
        note1 = base
        note2 = base+offset1
        note3 = note2+offset2

    elif(inversion==1):
        note1 = base+offset1
        note2 = note1+offset2
        note3 = base+12

    elif(inversion==2):
        note1 = base+offset1+offset2
        note2 = base+12
        note3 = note2+offset1

    return note1, note2, note3

print("Generating augmented triad chord MIDI files...")  # Changed to augmented

# Counter for generated files
file_count = 0

# Process all notes in all octaves without limiting
for q, j in zip(nameArray, range(24,109)):
    noteName = q
    base = j

    for h in range(0, 3):  # Loop through all 3 inversions
        inversion = h
        appendInversion = "-"+str(inversion)

        note1, note2, note3 = createTriad(inversion, base, offset1, offset2)
        degrees = [note1, note2, note3]

        track    = 0
        channel  = 0
        time_pos = 0    # In beats (renamed from 'time' to avoid conflict)
        duration = 1    # In beats
        tempo    = 60   # In BPM
        volume   = 100  # 0-127, as per the MIDI standard

        MyMIDI = MIDIFile(1)  # One track
        MyMIDI.addTempo(track, time_pos, tempo)

        # Generate chord (all notes played simultaneously)
        for pitch in degrees:
            MyMIDI.addNote(track, channel, pitch, time_pos, duration, volume)

        midiName = noteName+'-'+chord_type+'-chord'+appendInversion+".mid"
        midi_filename = os.path.join(midi_chord_dir, midiName)

        with open(midi_filename, "wb") as output_file:
            MyMIDI.writeFile(output_file)
            file_count += 1

            # Print progress every 10 files
            if file_count % 10 == 0:
                print(f"Generated {file_count} MIDI files...")

print(f"Completed! Generated {file_count} augmented triad chord MIDI files.")  # Changed to augmented

# Install timidity if not already installed
try:
    subprocess.run(['which', 'timidity'], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print("TiMidity++ is already installed.")
except subprocess.CalledProcessError:
    print("Installing TiMidity++...")
    subprocess.run(['apt-get', 'update'], check=True)
    subprocess.run(['apt-get', 'install', '-y', 'timidity'], check=True)
    print("TiMidity++ installed successfully.")

# Convert MIDI files to WAV with different noise levels
print("\nConverting MIDI files to WAV format with varying noise levels (0-5)...")
wav_count = 0

# Get all MIDI files in the chord directory
midi_files = [f for f in os.listdir(midi_chord_dir) if f.endswith('.mid')]
total_midi_files = len(midi_files)
total_conversions = total_midi_files * 6  # 6 versions of each file (noise levels 0-5)

for idx, midi_file in enumerate(midi_files):
    midi_path = os.path.join(midi_chord_dir, midi_file)
    base_name = Path(midi_file).stem

    print(f"Processing {idx+1}/{total_midi_files}: {midi_file}")

    # First, convert to WAV without noise (temp file)
    temp_wav_path = os.path.join('/tmp', f"{base_name}_temp.wav")

    try:
        # Run timidity with a timeout to prevent hanging
        # Increase the volume of the MIDI files when converting to make them more prominent
        timidity_process = subprocess.run(
            ['timidity', midi_path, '-Ow1', '-s', '16kHz', '-A', '200', '-o', temp_wav_path],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE,
            timeout=10  # Set a 10-second timeout for conversion
        )

        # Check if the temp file was created
        if not os.path.exists(temp_wav_path):
            print(f"Warning: Failed to create WAV file for {midi_file}")
            continue

        # Read the WAV file
        sample_rate, audio_data = wavfile.read(temp_wav_path)

        # Create versions with different noise levels
        for noise_level in range(6):  # 0 to 5
            # Add appropriate level of noise
            if noise_level == 0:
                # No noise for level 0
                noisy_audio = audio_data
            else:
                # Add increasing levels of noise
                noisy_audio = add_noise(audio_data, noise_level)

            # Save to final WAV file with noise level in filename
            wav_file = os.path.join(wav_chord_dir, f"{base_name}_noise{noise_level}.wav")
            wavfile.write(wav_file, sample_rate, noisy_audio)
            wav_count += 1

        # Success message for this file
        print(f"Created 6 variations with different noise levels for {midi_file}")

    except subprocess.TimeoutExpired:
        print(f"Warning: Conversion timeout for {midi_file}")
    except Exception as e:
        print(f"Error processing {midi_file}: {str(e)}")

    # Clean up temp file
    if os.path.exists(temp_wav_path):
        os.remove(temp_wav_path)

    # Add a small delay to prevent overwhelming the system
    time_module.sleep(0.1)  # Using the renamed time module

print(f"\nAll done! Generated {file_count} augmented triad chord MIDI files and {wav_count} WAV files with varying noise levels.")  # Changed to augmented
print(f"MIDI files are in: {midi_chord_dir}")
print(f"WAV files are in: {wav_chord_dir}")